# Synthesize a research paper
## Workflow
+ Define domain<br>
+ Build corpus<br>
+ Create embeddings model<br>
+ Perform WhiteSpace Analysis<br>
+ Identify best area for analysis<br>
+ Suggest topics for research<br>
+ Synthesize research paper content on selected topic

## Import Libraries

In [12]:
import os
import re
import string
import time

from arxiv import SortCriterion, SortOrder, Client, Search
import csv

import openai

import pdfplumber
import pytesseract
import fitz #pip install pymupdf
from PIL import Image
import io
import cv2
import numpy as np

from gensim.models.fasttext import FastText
import fasttext.util

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

from sklearn.decomposition import PCA
# pip install umap-learn
import umap

import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage
from bokeh.plotting import figure, output_notebook, show
import pandas as pd

# Inputs

In [2]:
# Define domain
domain = "petroleum engineering"
# misc definitions
corpus_directory = "./corpus"
dimensions = 100
max_papers = 100

In [3]:
 # Azure Instance
openai.api_key = ""
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

In [4]:
# Set the path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract' 
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
stemmer = WordNetLemmatizer()
# required data and model downloads
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
en_stop = set(nltk.corpus.stopwords.words('english'))

# define model training parameters
# the number of dimensions is the number of vectors to create that represent a word
embedding_size = dimensions
window_size = 40
min_word = 3
down_sampling = 1e-2
min_ngrams = 3
max_ngrams = 6
enrich_vectors = 1

train_filename = domain.replace(" ", "_")+"_train.txt"
VECTORS_FILEPATH = "cc.en."+str(dimensions)+".vec"
MODEL_NAME = domain.replace(" ", "_")+"_model.bin"
PRETRAINED_MODEL = "cc.en."+str(dimensions)+".bin"

[nltk_data] Downloading package wordnet to /home/titan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/titan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/titan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/titan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/titan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/titan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Helper functions

In [5]:
# create a function to generate key words for research in the defined domain
def generate_research_keywords_for_domain(domain:str)->str:
    prompt = "Create comma seperated key words for research queries in the "+domain+" domain:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.20,
            max_tokens=500
        )
    result = response.choices[0].text.strip()
    return result

def string_to_list(text:str)->str:
    """
    Converts a string of comma-separated values into a list of individual elements.
    """
    # Split the string into individual values using the comma separator
    values = text.split(',')
    
    # Remove any whitespace from the values
    values = [value.strip().replace(".", "") for value in values]
    
    # Return the list of values
    return values

def create_directory_if_not_exists(path:str):
    """
    Creates a directory at the specified path if it does not already exist.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return

# create a function to save papers by removing special characters from the file name
def clean_title(title:str):
    title = title.replace(" ", "_")
    title = title.replace("\\", "")
    title = title.replace("/", "")
    return title

def file_exists(path:str):
    """
    Checks if a file exists at the specified path.
    Returns True if the file exists, False otherwise.
    """
    return os.path.exists(path) and os.path.isfile(path)

def get_typed_files_in_directory(directory:str, file_type:str) -> list:
    """
    Returns a list of all files in the specified directory.
    """
    # Initialize an empty list to hold the PDF files
    typed_files = []

    # Loop over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file has a .pdf extension
        if filename.lower().endswith(f".{file_type.lower()}"):
            # If it's a PDF file, add the full path to the list
            typed_files.append(os.path.join(directory, filename))

    # Return the list of PDF files
    return typed_files

def write_or_append_file(filepath, text):
    """
    Writes the specified text to a file, creating the file if it does not exist,
    or appends the text to the end of the file if it already exists.
    """
    mode = "a" if os.path.exists(filepath) else "w"
    with open(filepath, mode) as f:
        f.write(text)
        
def extract_text_from_pdf(filepath:str)->str:
    """
    Extracts text from a PDF file, using OCR if the PDF contains images.
    """
    # Open the PDF file using pdfplumber
    with pdfplumber.open(filepath) as pdf:
        # Initialize an empty string to hold the extracted text
        text = ""

        # Loop over all pages in the PDF
        for page in pdf.pages:
            # Extract the page text using pdfplumber
            page_text = page.extract_text()

            # If the page text is None, the page may contain images, so use OCR to extract the text
            if page_text is None:
                # Convert the page image to grayscale and apply thresholding to remove noise
                page_image = page.to_image(resolution=150)
                page_image = page_image.convert('L')
                page_image = page_image.point(lambda x: 0 if x < 180 else 255, '1')

                # Use pytesseract to extract the text from the image
                page_text = pytesseract.image_to_string(page_image)

            # Add the page text to the overall text
            text += page_text

    # Return the extracted text
    return text

def rotate_image(image, angle):
    """
    Rotate an image by a given angle.

    Args:
        image (np.array): The image to rotate.
        angle (float): The angle to rotate the image by.

    Returns:
        np.array: The rotated image.
    """
    rows, cols = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, rotation_matrix, (cols, rows))

def correct_skew(image):
    """
    Correct skew in an image.

    Args:
        image (np.array): The image to correct skew for.

    Returns:
        np.array: The image with corrected skew.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    return rotate_image(image, angle)

def extract_text_from_pdf(pdf_path):
    """
    Extract text from an image-based PDF using the pymupdf library.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text from the PDF.
    """
    # Open the PDF file
    doc = fitz.open(pdf_path)

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    # Loop through each page in the PDF
    for page in doc:
        # Extract text from the page and append it to the extracted_text string
        extracted_text += page.get_text("text")

    # Close the PDF file
    doc.close()

    return extracted_text

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        # standardize white space
        document = re.sub(r'\s+', ' ', document)
        # Removing linebreaks '\n'
        document = re.sub(r'[\r\n]+', '', document)
        
        ## remove digits with regular expression
        document = re.sub(r'\d', ' ', document)
        #MM remove any patterns matching standard url format
        url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
        document = re.sub(url_pattern, ' ', document)
        #MM remove all non-ascii characters
        document = ''.join(character for character in document if ord(character)<128)
        #MM filter funny characters, if any.
        printable = set(string.printable)
        document = filter(lambda x: x in printable, document) #filter funny characters, if any.
        document = "".join(list(document))
        

        # Converting to Lowercase
        document = document.lower().translate(remove_punct_dict)
        
        

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        #Remove all short words that have a length < 3 character
        tokens = [word for word in tokens if len(word) > 3]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text
    
def loadModel(modelName):
    model = FastText.load(modelName)
    return model

## Build corpus
Here we will build a corpus of documents from ARXIV papers<br>
To search papers, generate a list of keywords of research areas in the defined domain

In [6]:
keywords = generate_research_keywords_for_domain(domain)
keywords = string_to_list(keywords)

In [7]:
keywords

['petroleum engineering',
 'oil exploration',
 'drilling',
 'reservoir engineering',
 'production engineering',
 'well completion',
 'well logging',
 'geology',
 'seismic surveying',
 'petroleum economics',
 'petroleum refining',
 'petroleum geology',
 'petroleum technology',
 'petroleum law',
 'petroleum safety',
 'petroleum management']

In [8]:
# download papers in each research area
create_directory_if_not_exists(corpus_directory)
for searchterm in keywords:
    # create arxiv client
    big_slow_client = Client(
      page_size = 1000,
      delay_seconds = 10,
      num_retries = 5
    )
    for result in big_slow_client.results(Search(query=searchterm, max_results = max_papers, sort_by = SortCriterion.Relevance, sort_order = SortOrder.Ascending)):
        filename = clean_title(result.title)+".pdf"
        filepath = os.path.join(corpus_directory, filename)
        try:
            if file_exists(filepath):
                print("downloaded paper: "+filepath)
            else:
                print("downloading paper: "+filepath)
                result.download_pdf(dirpath=corpus_directory, filename=filename)
                time.sleep(2)
        except:
            print("Error downloading paper: "+filename)
            time.sleep(10)
            continue

downloaded paper: ./corpus/Optimal_Economic_Operation_of_Liquid_Petroleum_Products_Pipeline_Systems.pdf
downloaded paper: ./corpus/Phase_behavior_in_petroleum_fluids_(A_Detailed_Descriptive_and_Illustrative_Account).pdf
downloaded paper: ./corpus/On_the_Determination_of_the_Solar_Rotation_Elements_i,_Ω_and_Period_using_Sunspot_Observations_by_Ruđer_Bošković_in_1777.pdf
downloaded paper: ./corpus/On_gray-box_modeling_for_virtual_flow_metering.pdf
downloaded paper: ./corpus/Reaction_Mechanisms_in_Petroleum:_From_Experimentation_to_Upgrading_and_Geological_Conditions.pdf
downloaded paper: ./corpus/Simulation_of_incompressible_two-phase_flow_in_porous_media_with_large_timesteps.pdf
downloaded paper: ./corpus/Multi-task_learning_for_virtual_flow_metering.pdf
downloaded paper: ./corpus/Forecasting_the_production_of_Distillate_Fuel_Oil_Refinery_and_Propane_Blender_net_production_by_using_Time_Series_Algorithms.pdf
downloaded paper: ./corpus/A_Decision_Support_System_for_Multi-target_Geosteeri

downloaded paper: ./corpus/Application_of_Probabilistic_Graphical_Models_in_Forecasting_Crude_Oil_Price.pdf
downloaded paper: ./corpus/Experimental_evaluation_of_a_silicone_oil_as_an_oxidation_inhibitor_for_magnesium_alloy_under_contact_sliding_at_elevated_temperatures.pdf
downloaded paper: ./corpus/Exploring_the_use_of_Transition_Path_Theory_in_building_an_oil_spill_prediction_scheme.pdf
downloaded paper: ./corpus/Exploration_of_Spanish_Olive_Oil_Quality_with_a_Miniaturized_Low-Cost_Fluorescence_Sensor_and_Machine_Learning_Techniques.pdf
downloaded paper: ./corpus/Olive_Oil_is_Made_of_Olives,_Baby_Oil_is_Made_for_Babies:_Interpreting_Noun_Compounds_using_Paraphrases_in_a_Neural_Model.pdf
downloaded paper: ./corpus/Stability_of_additive-free_water-in-oil_emulsions.pdf
downloaded paper: ./corpus/Pickering_emulsions_with_alpha-cyclodextrin_inclusions:_Structure_and_thermal_stability.pdf
downloaded paper: ./corpus/Physico-chemical_properties_extraction_from_the_fluorescence_spectrum_with_

downloaded paper: ./corpus/Probing_the_surface_of_synthetic_opals_with_the_vanadyl-containing_crude_oil_by_using_EPR_and_ENDOR_techniques.pdf
downloaded paper: ./corpus/Enhanced_dielectric_breakdown_performance_of_anatase_and_rutile_titania_based_nano-oils.pdf
downloaded paper: ./corpus/Dust_evolution,_a_global_view:_II._Top-down_branching,_nano-particle_fragmentation_and_the_mystery_of_the_diffuse_interstellar_band_carriers.pdf
downloaded paper: ./corpus/Single_beam_acoustical_tweezers_based_on_focused_beams:_A_numerical_analysis_of_2D_and_3D_trapping_capabilities.pdf
downloaded paper: ./corpus/Forecasting_the_abnormal_events_at_well_drilling_with_machine_learning.pdf
downloaded paper: ./corpus/Violent_music_vs_violence_and_music:_Drill_rap_and_violent_crime_in_London.pdf
downloaded paper: ./corpus/DREAMS:_Drilling_and_Extraction_Automated_System.pdf
downloaded paper: ./corpus/On_the_characterization_of_drilling_rotation_in_the_6-parameter_resultant_shell_theory.pdf
downloaded paper: 

downloaded paper: ./corpus/Maximal_Steered_Coherence_Protection_by_Quantum_Reservoir_Engineering.pdf
downloaded paper: ./corpus/Extracting_work_from_random_collisions:_A_model_of_a_quantum_heat_engine.pdf
downloaded paper: ./corpus/Designing_reservoirs_for_1t_decoherence_of_a_qubit.pdf
downloaded paper: ./corpus/Carnot's_theorem_for_nonthermal_stationary_reservoirs.pdf
downloaded paper: ./corpus/A_Micrometer-sized_Heat_Engine_Operating_Between_Bacterial_Reservoirs.pdf
downloaded paper: ./corpus/Efficiency_of_heat_engines_coupled_to_nonequilibrium_reservoirs.pdf
downloaded paper: ./corpus/Optimization_performance_of_quantum_Otto_heat_engines_and_refrigerators_with_squeezed_thermal_reservoirs.pdf
downloaded paper: ./corpus/Quantitative_supply_security_related_significance_measures_for_gas_reservoires.pdf
downloaded paper: ./corpus/Electromagnetically_Induced_Transparency_and_Quantum_Heat_Engines.pdf
downloaded paper: ./corpus/The_thermodynamics_governing_'endoreversible'_engines.pdf
down

downloaded paper: ./corpus/A_Business_Maturity_Model_of_Software_Product_Line_Engineering.pdf
downloaded paper: ./corpus/An_Architecture_Process_Maturity_Model_of_Software_Product_Line_Engineering.pdf
downloaded paper: ./corpus/Variability_and_Evolution_in_Systems_of_Systems.pdf
downloaded paper: ./corpus/Towards_a_Systems_Engineering_based_Automotive_Product_Engineering_Process.pdf
downloaded paper: ./corpus/Towards_a_Systems_Engineering_Essence.pdf
downloaded paper: ./corpus/Mind_the_Gap:_On_the_Relationship_Between_Automatically_Measured_and_Self-Reported_Productivity.pdf
downloaded paper: ./corpus/Resolving_code_smells_in_software_product_line_using_refactoring_and_reverse_engineering.pdf
downloaded paper: ./corpus/Fault-Tolerant_Dot-Product_Engines.pdf
downloaded paper: ./corpus/Data_Engineering_for_the_Analysis_of_Semiconductor_Manufacturing_Data.pdf
downloaded paper: ./corpus/Optimization_analysis_of_an_endoreversible_quantum_heat_engine_with_efficient_power_function.pdf
downloa

...

downloaded paper: ./corpus/Robust_Oil-spill_Forensics_and_Petroleum_Source_Differentiation_using_Quantized_Peak_Topography_Maps.pdf
downloaded paper: ./corpus/How_to_use_Big_Data_technologies_to_optimize_operations_in_Upstream_Petroleum_Industry.pdf
downloaded paper: ./corpus/Phase_behavior_in_petroleum_fluids_(A_Detailed_Descriptive_and_Illustrative_Account).pdf
downloaded paper: ./corpus/Multi-task_learning_for_virtual_flow_metering.pdf
downloaded paper: ./corpus/Vortex_flow_in_the_technology_of_radiation_wave_cracking_(RWC).pdf
downloaded paper: ./corpus/User_Association_in_User-Centric_Hybrid_VLCRF_Cell-Free_Massive_MIMO_Systems.pdf
downloaded paper: ./corpus/Industrial_applications_of_digital_rock_technology.pdf
downloaded paper: ./corpus/The_Non-Organic_Theory_of_the_Genesis_of_Petroleum.pdf
downloaded paper: ./corpus/Impact_of_Recent_Discoveries_on_Petroleum_and_Natural_Gas_Exploration:_Emphasis_on_India.pdf
downloaded paper: ./corpus/Petroleum_prices_prediction_using_data_minin

downloaded paper: ./corpus/Phase_behavior_in_petroleum_fluids_(A_Detailed_Descriptive_and_Illustrative_Account).pdf
downloaded paper: ./corpus/The_Evolution_of_Multicomponent_Systems_at_High_Pressures:_VI._The_Thermodynamic_Stability_of_the_Hydrogen-Carbon_System:_The_Genesis_of_Hydrocarbons_and_the_Origin_of_Petroleum.pdf
downloaded paper: ./corpus/Evaluation_of_Gaussian_approximations_for_data_assimilation_in_reservoir_models.pdf
downloaded paper: ./corpus/Critical_bursts_in_filtration.pdf
downloaded paper: ./corpus/The_Non-Organic_Theory_of_the_Genesis_of_Petroleum.pdf
downloaded paper: ./corpus/Impact_of_Recent_Discoveries_on_Petroleum_and_Natural_Gas_Exploration:_Emphasis_on_India.pdf
downloaded paper: ./corpus/Petroleum_prices_prediction_using_data_mining_techniques_--_A_Review.pdf
downloaded paper: ./corpus/Central_Schemes_for_Porous_Media_Flows.pdf
downloaded paper: ./corpus/On_the_upstream_mobility_scheme_for_two-phase_flow_in_porous_media.pdf
downloaded paper: ./corpus/A_nume

downloaded paper: ./corpus/Phase_behavior_in_petroleum_fluids_(A_Detailed_Descriptive_and_Illustrative_Account).pdf
downloaded paper: ./corpus/Thermo-kinetic_explosions:_safety_first_or_safety_last?.pdf
downloaded paper: ./corpus/The_Non-Organic_Theory_of_the_Genesis_of_Petroleum.pdf
downloaded paper: ./corpus/Impact_of_Recent_Discoveries_on_Petroleum_and_Natural_Gas_Exploration:_Emphasis_on_India.pdf
downloaded paper: ./corpus/Petroleum_prices_prediction_using_data_mining_techniques_--_A_Review.pdf
downloaded paper: ./corpus/The_application_of_high-resolution_3D_seismic_data_to_model_the_distribution_of_mechanical_and_hydrogeological_properties_of_a_potential_host_rock_for_the_deep_storage_of_radioactive_waste_in_France.pdf
downloaded paper: ./corpus/Deghosting_of_Ocean_Bottom_Cable_Data:Two_approaches.pdf
downloaded paper: ./corpus/A_Dynamic_Sustainable_Competitive_Petroleum_Supply_Chain_Model_for_Various_Stakeholders_with_Shared_Facilities.pdf
downloaded paper: ./corpus/Analytical_I

downloading paper: ./corpus/Learning-based_Initialization_Strategy_for_Safety_of_Multi-Vehicle_Systems.pdf
downloading paper: ./corpus/Joint_Synthesis_of_Safety_Certificate_and_Safe_Control_Policy_using_Constrained_Reinforcement_Learning.pdf
downloading paper: ./corpus/Strategy_to_Increase_the_Safety_of_a_DNN-based_Perception_for_HAD_Systems.pdf
downloading paper: ./corpus/Small-Gain_Theorem_for_Safety_Verification_of_Interconnected_Systems.pdf
downloading paper: ./corpus/Product_safety_idioms:_a_method_for_building_causal_Bayesian_networks_for_product_safety_and_risk_assessment.pdf
downloading paper: ./corpus/Quantifying_the_Safety_of_Trajectories_using_Peak-Minimizing_Control.pdf
downloaded paper: ./corpus/Safety_Practice_and_its_Practitioners:_Exploring_a_Diverse_Profession.pdf
downloaded paper: ./corpus/On_the_efficacy_of_safety-related_software_standards.pdf
downloading paper: ./corpus/Do_you_feel_safe_with_your_robot?_Factors_Influencing_Perceived_Safety_in_Human-Robot_Interactio

downloading paper: ./corpus/Domain-partitioned_element_management_systems_employing_mobile_agents_for_distributed_network_management.pdf
downloaded paper: ./corpus/Machine_Learning-Based_Soft_Sensors_for_Vacuum_Distillation_Unit.pdf
downloaded paper: ./corpus/Long-Term_Optimal_Delivery_Planning_for_Replacing_the_Liquefied_Petroleum_Gas_Cylinder.pdf
downloading paper: ./corpus/Datom:_Towards_modular_data_management.pdf
downloaded paper: ./corpus/Central_Schemes_for_Porous_Media_Flows.pdf
downloaded paper: ./corpus/On_the_upstream_mobility_scheme_for_two-phase_flow_in_porous_media.pdf
downloaded paper: ./corpus/Polysiloxane_surfactants_for_the_dispersion_of_carbon_nanotubes_in_non-polar_organic_solvents.pdf
downloaded paper: ./corpus/Dynamic_Decision_Making_for_Graphical_Models_Applied_to_Oil_Exploration.pdf
downloaded paper: ./corpus/Distributed_computing_of_Seismic_Imaging_Algorithms.pdf
downloaded paper: ./corpus/Titania_Prepared_by_Ball_Milling:_Its_Characterization_and_Application_a

In [9]:
# get a list of files downloaded to the directory to process
pdf_file_list = get_typed_files_in_directory(corpus_directory, "pdf")

In [10]:
# extract the text from the pdf files
for file in pdf_file_list:
    try:
        print(f"processing file {file}")
        text = extract_text_from_pdf(file)
        if text is not None and len(text) > 10:
            write_or_append_file(train_filename, preprocess_text(text))
        else:
            print(f"non text extract from file {file}")
    except:
        print(f"error with text extract from file {file}")

processing file ./corpus/Three-level_laser_heat_engine_at_optimal_performance_with_ecological_function.pdf
processing file ./corpus/Convergence_and_completeness_for_square-well_Stark_resonant_state_expansions.pdf
processing file ./corpus/Cellular_Memristive-Output_Reservoir_(CMOR).pdf
processing file ./corpus/A_new_design_strategy_based_on_a_deterministic_definition_of_the_seismic_input_to_overcome_the_limits_of_design_procedures_based_on_probabilistic_approaches.pdf
processing file ./corpus/Distributed_Kerr_Nonlinearity_in_a_Coherent_All-Optical_Fiber-Ring_Reservoir_Computer.pdf
processing file ./corpus/Completion_Time_in_Multi-Access_Channel:_An_Information_Theoretic_Perspective.pdf
processing file ./corpus/Machine_Learning_for_Software_Engineering:_A_Tertiary_Study.pdf
processing file ./corpus/Noncommutative_Schur-type_products_and_their_Schoenberg_theorem.pdf
processing file ./corpus/INSiDER:_Incorporation_of_system_and_safety_analysis_models_using_a_dedicated_reference_model.pdf
p

processing file ./corpus/Dissolved_gas_monitoring_probe_without_liquid-gas_separation_under_strong_electromagnetic_interference.pdf
processing file ./corpus/A_100%_Renewable_Energy_System:_Enabling_Zero_CO2_Emission_Offshore_Platforms.pdf
processing file ./corpus/Environmental_Factors_Influencing_Individual_Decision-Making_Behavior_in_Software_Project:_A_Systematic_Literature_Review.pdf
processing file ./corpus/Parametrisation_in_electrostatic_DPD_Dynamics_and_Applications.pdf
processing file ./corpus/Work_producing_reservoirs:_Stochastic_thermodynamics_with_generalized_Gibbs_ensembles.pdf
processing file ./corpus/Analyzing_the_effect_of_local_rounding_error_propagation_on_the_maximal_attainable_accuracy_of_the_pipelined_Conjugate_Gradient_method.pdf
processing file ./corpus/Dynamics_of_entangled_states_in_squeezed_reservoirs.pdf
processing file ./corpus/Fracture_Propagation_Driven_by_Fluid_Outflow_from_a_Low-permeability_Aquifer.pdf
processing file ./corpus/On_gray-box_modeling_for_vi

processing file ./corpus/Normative_Engineering_Risk_Management_Systems.pdf
processing file ./corpus/Discrete-time_signatures_and_randomness_in_reservoir_computing.pdf
processing file ./corpus/Critical_behavior_for_mixed_site-bond_directed_percolation.pdf
processing file ./corpus/The_Galactic_origin_for_the_borders_in_the_Earth_history.pdf
processing file ./corpus/The_Power_of_Language:_Understanding_Sentiment_Towards_the_Climate_Emergency_using_Twitter_Data.pdf
processing file ./corpus/Modeling_the_impact_of_dilution_on_the_microbial_degradation_time_of_dispersed_oil_in_marine_environments.pdf
processing file ./corpus/Meet-completions_and_ordered_domain_algebras.pdf
processing file ./corpus/Assessing_the_Impact_of_Offshore_Wind_Siting_Strategies_on_the_Design_of_the_European_Power_System.pdf
processing file ./corpus/Wormhole_formation_in_dissolving_fractures.pdf
processing file ./corpus/Numerical_computation_of_stress-permeability_relationships_of_fracture_networks_in_a_shale_rock.pdf


processing file ./corpus/Products_of_manifolds_with_fibered_corners.pdf
processing file ./corpus/Universal_Graphs_at_$aleph_{ω_1+1}$_and_Set-theoretic_Geology.pdf
processing file ./corpus/Forecasting_the_term_structure_of_crude_oil_futures_prices_with_neural_networks.pdf
processing file ./corpus/Extended_source_imaging,_a_unifying_framework_for_seismic_&_medical_imaging.pdf
processing file ./corpus/Review_of_medical_data_analysis_based_on_spiking_neural_networks.pdf
processing file ./corpus/Mutual_Information_and_the_Edge_of_Chaos_in_Reservoir_Computers.pdf
processing file ./corpus/On_the_Truncated_Pareto_Distribution_with_applications.pdf
processing file ./corpus/Task-agnostic_Exploration_in_Reinforcement_Learning.pdf
processing file ./corpus/A_Non-standard_Model_for_Microbial_Enhanced_Oil_Recovery_Including_the_Oil-water_Interfacial_Area.pdf
processing file ./corpus/Efficient_depth_extrapolation_of_waves_in_elastic_isotropic_media.pdf
processing file ./corpus/Do_Software_Languages_En

processing file ./corpus/A_Separated_Cohomologically_Complete_Module_is_Complete.pdf
processing file ./corpus/Generalizing_Interactive_Backpropagating_Refinement_for_Dense_Prediction.pdf
processing file ./corpus/Square_ice_in_graphene_nanocapillaries.pdf
processing file ./corpus/Wave_durationpersistence_statistics,_recording_interval,_and_fractal_dimension.pdf
processing file ./corpus/Learning_Exploration_Policies_for_Navigation.pdf
processing file ./corpus/New_characterizations_of_the_region_of_complete_localization_for_random_Schrödinger_operators.pdf
processing file ./corpus/ReinBo:_Machine_Learning_pipeline_search_and_configuration_with_Bayesian_Optimization_embedded_Reinforcement_Learning.pdf
processing file ./corpus/Sparse_SPN:_Depth_Completion_from_Sparse_Keypoints.pdf
processing file ./corpus/Singular_degenerate_problems_and_applications.pdf
processing file ./corpus/The_Non-Organic_Theory_of_the_Genesis_of_Petroleum.pdf
processing file ./corpus/A_Predictive_Model_for_Oil_Market

...

processing file ./corpus/Cultural_Barriers_to_Software_Productivity_Practices_at_Los_Alamos.pdf
processing file ./corpus/A_concise_review_on_THGEM_detectors.pdf
processing file ./corpus/Recognizing_well-dominated_graphs_is_coNP-complete.pdf
processing file ./corpus/Young_alpha-enriched_giant_stars_in_the_solar_neighbourhood.pdf
processing file ./corpus/The_classification_of_smooth_well-formed_Fano_weighted_complete_intersections.pdf
processing file ./corpus/Complete_sets_and_completion_of_sets_in_Banach_spaces.pdf
processing file ./corpus/Modification_of_the_trapped_field_in_bulk_high-temperature_superconductors_as_a_result_of_the_drilling_of_a_pattern_of_artificial_columnar_holes.pdf
processing file ./corpus/An_End-to-End_Two-Phase_Deep_Learning-Based_workflow_to_Segment_Man-made_Objects_Around_Reservoirs.pdf
processing file ./corpus/Oil-US_Stock_Market_Nexus:_Some_insights_about_the_New_Coronavirus_Crisis.pdf
processing file ./corpus/The_squeezed_thermal_reservoir_as_a_generalized_eq

processing file ./corpus/Efficient_p-multigrid_spectral_element_model_for_water_waves_and_marine_offshore_structures.pdf
processing file ./corpus/Characterizing_Uncertainty_in_the_Visual_Text_Analysis_Pipeline.pdf
processing file ./corpus/Investigations_of_the_pi_N_total_cross_sections_at_high_energies_using_new_FESR:_log_nu_or_(log_nu)^2.pdf
processing file ./corpus/Covariant-Contravariant_Refinement_Modal_$μ$-calculus.pdf
processing file ./corpus/Software_Uncertainty_in_Integrated_Environmental_Modelling:_the_role_of_Semantics_and_Open_Science.pdf
processing file ./corpus/The_refined_BPS_index_from_stable_pair_invariants.pdf
processing file ./corpus/Towards_Personalized_Preprocessing_Pipeline_Search.pdf
processing file ./corpus/Unsupervised_seismic_facies_classification_using_deep_convolutional_autoencoder.pdf
processing file ./corpus/Sets_completely_separated_by_functions_in_Bishop_Set_Theory.pdf
processing file ./corpus/Multiscale_simulation_of_injection-induced_fracture_slip_and_w

processing file ./corpus/Optimizing_Quantum_Noise-induced_Reservoir_Computing_for_Nonlinear_and_Chaotic_Time_Series_Prediction.pdf
processing file ./corpus/Solving_non-Markovian_open_quantum_systems_with_multi-channel_reservoir_coupling.pdf
processing file ./corpus/Decoherence_suppression_of_a_dissipative_qubit_by_the_non-Markovian_effect.pdf
processing file ./corpus/The_inverse_problem_in_Seismology._Seismic_moment_and_energy_of_earthquakes._Seismic_hyperbola.pdf
processing file ./corpus/Exploring_the_near-surface_at_the_lunar_South_Pole_with_geophysical_tools.pdf
processing file ./corpus/On_the_Spaces_and_Dimensions_of_Geographical_Systems.pdf
processing file ./corpus/Existence_of_complete_Lyapunov_functions_for_semiflows_on_separable_metric_spaces.pdf
processing file ./corpus/On_completely_regular_codes_with_minimum_eigenvalue_in_geometric_graphs.pdf
processing file ./corpus/Macroeconomic_performance_of_oil_price_shocks_in_Russia.pdf
processing file ./corpus/Polarization_Modeling_an

processing file ./corpus/A_numerical_exploration_of_signal_detector_arrangement_in_a_spin-wave_reservoir_computing_device.pdf
processing file ./corpus/Memory_Safety_Preservation_for_WebAssembly.pdf
processing file ./corpus/On_the_Correlation_Between_Atomic_Gas_and_Bars_in_Galaxies.pdf
processing file ./corpus/Does_Global_seismic_energy_release_increase?_An_analysis_based_on_the_Lithospheric_Seismic_Energy_Flow_Model_(LSEFM)._The_case_of_mega_-_earthquakes_(M_>_9).pdf
processing file ./corpus/Repair_Pipelining_for_Erasure-Coded_Storage:_Algorithms_and_Evaluation.pdf
processing file ./corpus/Thompson_Sampling_for_(Combinatorial)_Pure_Exploration.pdf
processing file ./corpus/Beyond_NGS_data_sharing_and_towards_open_science.pdf
processing file ./corpus/Capillary_fracture_of_ultrasoft_gels:_heterogeneity_and_delayed_nucleation.pdf
processing file ./corpus/Probing_the_surface_of_synthetic_opals_with_the_vanadyl-containing_crude_oil_by_using_EPR_and_ENDOR_techniques.pdf
processing file ./corp

processing file ./corpus/On_the_Containment_Problem_for_Linear_Sets.pdf
processing file ./corpus/Exact_3D_seismic_data_reconstruction_using_Tubal-Alt-Min_algorithm.pdf
processing file ./corpus/Deep_recommender_engine_based_on_efficient_product_embeddings_neural_pipeline.pdf
processing file ./corpus/Self-Supervised_Exploration_via_Disagreement.pdf
processing file ./corpus/Critical_bursts_in_filtration.pdf
processing file ./corpus/Model-driven_Engineering_of_Safety_and_Security_Systems:_A_Systematic_Mapping_Study.pdf
processing file ./corpus/Designing_biodegradable_surfactants_and_effective_biomolecules_with_dissipative_particle_dynamics.pdf
processing file ./corpus/The_MKID_Science_Data_Pipeline.pdf
processing file ./corpus/Pseudo-2D_RANS:_A_LiDAR-driven_mid-fidelity_model_for_simulations_of_wind_farm_flows.pdf
processing file ./corpus/On_Edge-Partitioning_of_Complete_Geometric_Graphs_into_Plane_Trees.pdf
processing file ./corpus/Perspectives_of_cross_correlation_in_seismic_monitoring_a

In [6]:
# Need to convert the .bin file to a .vec file
# this takes a while
isExist = os.path.exists(VECTORS_FILEPATH)
if not isExist:
    fasttext.util.download_model('en', if_exists='ignore')  # English
    ft = fasttext.load_model('cc.en.300.bin')
    if dimensions != 300:
        ft = fasttext.util.reduce_model(ft, dimensions)
    
    lines=[]
    
    # get all words from model
    words = ft.get_words()
    
    with open(VECTORS_FILEPATH,'w') as file_out:
        
        # the first line must contain number of total words and vector dimension
        file_out.write(str(len(words)) + " " + str(ft.get_dimension()) + "\n")
    
        # line by line, you append vectors to VEC file
        for w in words:
            v = ft.get_word_vector(w)
            vstr = ""
            for vi in v:
                vstr += " " + str(vi)
            try:
                file_out.write(w + vstr+'\n')
            except:
                pass

In [7]:
# train (fine-tune) the model
model_ft = fasttext.train_unsupervised(input=train_filename, lr=.1, epoch=200, bucket=200000,
                             minn=3, maxn=6, wordNgrams=max_ngrams, dim=embedding_size, 
                             pretrainedVectors=VECTORS_FILEPATH, thread=6)

Read 9M words
Number of words:  54842
Number of labels: 0
Progress:  62.3% words/sec/thread:   49261 lr:  0.037652 avg.loss:  0.128338 ETA:   0h47m41s  4.7% words/sec/thread:   52199 lr:  0.095320 avg.loss:  0.567002 ETA:   1h53m56sm24s  7.5% words/sec/thread:   49259 lr:  0.092519 avg.loss:  0.520770 ETA:   1h57m11sh57m48s 10.6% words/sec/thread:   48833 lr:  0.089372 avg.loss:  0.423819 ETA:   1h54m11s  49297 lr:  0.082986 avg.loss:  0.332546 ETA:   1h45m 2s  49223 lr:  0.082000 avg.loss:  0.323680 ETA:   1h43m56s 18.5% words/sec/thread:   49147 lr:  0.081516 avg.loss:  0.319638 ETA:   1h43m29s 18.5% words/sec/thread:   49139 lr:  0.081450 avg.loss:  0.319041 ETA:   1h43m25s 19.3% words/sec/thread:   49158 lr:  0.080654 avg.loss:  0.312827 ETA:   1h42m22s 21.9% words/sec/thread:   49295 lr:  0.078063 avg.loss:  0.294919 ETA:   1h38m48s  49292 lr:  0.077334 avg.loss:  0.290491 ETA:   1h37m53s ETA:   1h37m40s  1h37m34s 23.1% words/sec/thread:   49216 lr:  0.076857 avg.loss:  0.287712 E

In [8]:
model_ft.save_model(MODEL_NAME)

In [9]:
model_ft = fasttext.load_model(MODEL_NAME)
words = model_ft.get_words()
# Get the word vectors
vectors = [model_ft.get_word_vector(word) for word in words]

In [10]:
# Reduce dimensions using UMAP
umap_reducer = umap.UMAP(n_components=2, random_state=42)
reduced_vectors = umap_reducer.fit_transform(vectors)

In [13]:
# Plot the words using the 2-dimensional vectors from UMAP
plt.figure(figsize=(12, 12))

for i, word in enumerate(words):
    plt.scatter(reduced_vectors[i, 0], reduced_vectors[i, 1])
    plt.annotate(word, (reduced_vectors[i, 0], reduced_vectors[i, 1]), fontsize=10)

plt.title('2D Visualization of Word Vectors using UMAP')
plt.xlabel('UMAP X')
plt.ylabel('UMAP Y')
plt.show()

In [14]:
# Create a dataframe for Datashader
data = pd.DataFrame(reduced_vectors, columns=['x', 'y'])
data['word'] = words

# Create a Datashader image
def create_image(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(data, 'x', 'y', ds.count())
    img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

# Create a Bokeh plot
output_notebook()
p = figure(width=800, height=800, x_range=(-10, 10), y_range=(-10, 10),
           toolbar_location="above", tools="pan,wheel_zoom,box_zoom,reset")
p.axis.visible = False
InteractiveImage(p, create_image)
show(p)